In [186]:
from chembl_webresource_client.new_client import new_client
from chembl_webresource_client.query_set import QuerySet
import pandas as pd

In [188]:
def QuerySetMWRangeFilter(less_limit: int = 0, greater_limit: int = 12_546_42) -> QuerySet:
    """      
    QuerySet Molecular Weight Range Filter - функция, которая выполняет фильтрацию по базе ChEMBL
    по диапазону молекулярного веса (обе границы включительно)

    Args:
        less_limit (int): нижняя граница
        greater_limit (int): верхняя граница

    Raises:
        ValueError: если границы меньше являются отриц. числами
        ValueError: если, верхняя граница больше нижней

    Returns:
        QuerySet: список объектов заданной модели
    """

    if greater_limit < 0 or less_limit < 0:
        raise ValueError(
            "QuerySetMWRangeFilter: limits should be greater zero")

    if greater_limit < less_limit:
        raise ValueError(
            "QuerySetMWRangeFilter: greater_limit should be greater than less_limit")

    
    return new_client.molecule.filter(molecule_properties__mw_freebase__lte=greater_limit, molecule_properties__mw_freebase__gte=less_limit) # type: ignore


In [189]:
def ExpandedFLDF(data: pd.DataFrame) -> pd.DataFrame:
    """
    ExpandedFromListsDataFrame - функция, которая переписывает словари и списки словарей 
    в таблице в отдельные столбцы

    Args:
        data (pd.DataFrame): изначальная таблица

    Returns:
        pd.DataFrame: "раскрытая" таблица
    """

    exposed_data = pd.DataFrame({
        #! cross_references
        'xref_id':                     data['cross_references'].apply(lambda x: [d['xref_id'] for d in x] if x else []),
        'xref_name':                   data['cross_references'].apply(lambda x: [d['xref_name'] for d in x] if x else []),
        'xref_src':                    data['cross_references'].apply(lambda x: [d['xref_src'] for d in x] if x else []),
        #! molecule_hierarchy
        'active_chembl_id':            [item['active_chembl_id'] if isinstance(item, dict) else None for item in data['molecule_hierarchy']],
        'molecule_chembl_id':          [item['molecule_chembl_id'] if isinstance(item, dict) else None for item in data['molecule_hierarchy']],
        'parent_chembl_id':            [item['parent_chembl_id'] if isinstance(item, dict) else None for item in data['molecule_hierarchy']],
        #! molecule_properties
        'alogp':                       [item['alogp'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'aromatic_rings':              [item['aromatic_rings'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'cx_logd':                     [item['cx_logd'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'cx_logp':                     [item['cx_logp'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'cx_most_apka':                [item['cx_most_apka'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'cx_most_bpka':                [item['cx_most_bpka'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'full_molformula':             [item['full_molformula'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'full_mwt':                    [item['full_mwt'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'hba':                         [item['hba'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'hba_lipinski':                [item['hba_lipinski'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'hbd':                         [item['hbd'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'hbd_lipinski':                [item['hbd_lipinski'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'heavy_atoms':                 [item['heavy_atoms'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'molecular_species':           [item['molecular_species'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'mw_freebase':                 [item['mw_freebase'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'mw_monoisotopic':             [item['mw_monoisotopic'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'np_likeness_score':           [item['np_likeness_score'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'num_lipinski_ro5_violations': [item['num_lipinski_ro5_violations'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'num_ro5_violations':          [item['num_ro5_violations'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'psa':                         [item['psa'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'qed_weighted':                [item['qed_weighted'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'ro3_pass':                    [item['ro3_pass'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        'rtb':                         [item['rtb'] if isinstance(item, dict) else None for item in data['molecule_properties']],
        #! molecule_structures
        'canonical_smiles':            [item['canonical_smiles'] if isinstance(item, dict) else None for item in data['molecule_structures']],
        # 'molfile':                   [item['molfile'] if isinstance(item, dict) else None for item in data['molecule_structures']], - какая-то стрёмная хрень с RDKit
        'standard_inchi':              [item['standard_inchi'] if isinstance(item, dict) else None for item in data['molecule_structures']],
        'standard_inchi_key':          [item['standard_inchi_key'] if isinstance(item, dict) else None for item in data['molecule_structures']],
        #! molecule_synonyms
        'molecule_synonym':            data['molecule_synonyms'].apply(lambda x: [d['molecule_synonym'] for d in x] if x else []),
        'syn_type':                    data['molecule_synonyms'].apply(lambda x: [d['syn_type'] for d in x] if x else []),
        'synonyms':                    data['molecule_synonyms'].apply(lambda x: [d['synonyms'] for d in x] if x else []),
    })

    data = data.drop(['cross_references', 'molecule_hierarchy',
                     'molecule_properties', 'molecule_structures', 'molecule_synonyms'], axis=1)

    return pd.concat([data, exposed_data], axis=1)


In [190]:
under_100_mw_mols: QuerySet = QuerySetMWRangeFilter(0, 100)

print(len(under_100_mw_mols))  # type: ignore

# Pandas
data = pd.DataFrame(under_100_mw_mols)  # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_100_mw_mols_output.csv", index=False)
# data


# Tabulate
# print(tabulate(under_100_mw_mols, headers="keys"))
# print(type(tabulate(under_100_mw_mols, headers="keys")))


1042


In [45]:
under_200_mw_mols: QuerySet = QuerySetMWRangeFilter(100, 200)

print(len(under_200_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_200_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_200_mw_mols_output.csv", index=False)
# data

43591


In [46]:
under_300_mw_mols: QuerySet = QuerySetMWRangeFilter(200, 300)

print(len(under_300_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_300_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_300_mw_mols_output.csv", index=False)

382494


In [47]:
under_400_mw_mols: QuerySet = QuerySetMWRangeFilter(300, 400)

print(len(under_400_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_400_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_400_mw_mols_output.csv", index=False)
# data

862780


In [48]:
under_500_mw_mols: QuerySet = QuerySetMWRangeFilter(400, 500)

print(len(under_500_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_500_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_500_mw_mols_output.csv", index=False)
# data

662709


In [49]:
under_600_mw_mols: QuerySet = QuerySetMWRangeFilter(500, 600)

print(len(under_600_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_600_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_600_mw_mols_output.csv", index=False)
# data

256208


In [50]:
under_700_mw_mols: QuerySet = QuerySetMWRangeFilter(600, 700)

print(len(under_700_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_700_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_700_mw_mols_output.csv", index=False)
# data

85072


In [51]:
under_800_mw_mols: QuerySet = QuerySetMWRangeFilter(700, 800)

print(len(under_800_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_800_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_800_mw_mols_output.csv", index=False)
# data

36280


In [52]:
under_900_mw_mols: QuerySet = QuerySetMWRangeFilter(800, 900)

print(len(under_900_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_900_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_900_mw_mols_output.csv", index=False)
# data

20907


In [54]:
under_1000_mw_mols: QuerySet = QuerySetMWRangeFilter(900, 1000)

print(len(under_1000_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(under_1000_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/under_1000_mw_mols_output.csv", index=False)
# data

12910


KeyboardInterrupt: 

In [53]:
above_1000_mw_mols: QuerySet = QuerySetMWRangeFilter(1000)

print(len(above_1000_mw_mols)) # type: ignore

# Pandas
data = pd.DataFrame(above_1000_mw_mols) # type: ignore
data = ExpandedFLDF(data)
data.to_csv("results/above_1000_mw_mols_output.csv", index=False)
# data

49007
